# EDA - Wisconsin Breast Cancer Dataset

Ce notebook documente l'exploration de donnees pour le projet de detection du cancer du sein.

Objectifs:
- verifier la qualite des donnees,
- analyser la distribution des classes,
- inspecter les valeurs manquantes,
- observer les correlations des variables tabulaires.

In [4]:
from pathlib import Path

import pandas as pd
from IPython.display import display
import plotly.express as px

candidate_paths = [Path('data.csv'), Path('../data.csv')]
DATA_PATH = next((path for path in candidate_paths if path.exists()), None)
if DATA_PATH is None:
    searched = ', '.join(str(path.resolve()) for path in candidate_paths)
    raise FileNotFoundError(f'Dataset introuvable. Chemins testes: {searched}')

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)
print('Notebook pret. Dataset:', DATA_PATH.resolve())

Notebook pret. Dataset: D:\Deeplearning Project\data.csv


In [5]:
df = pd.read_csv(DATA_PATH)
print('Shape:', df.shape)
display(df.head(3))
display(df.dtypes.head(10))

Shape: (569, 33)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN


id                       int64
diagnosis               object
radius_mean            float64
texture_mean           float64
perimeter_mean         float64
area_mean              float64
smoothness_mean        float64
compactness_mean       float64
concavity_mean         float64
concave points_mean    float64
dtype: object

In [6]:
# Nettoyage minimal pour l'EDA
df_eda = df.copy()
df_eda.columns = [col.strip().replace(' ', '_') for col in df_eda.columns]
if 'Unnamed:_32' in df_eda.columns:
    df_eda = df_eda.drop(columns=['Unnamed:_32'])

target_col = 'diagnosis'
id_col = 'id'

print('Colonnes apres nettoyage:', len(df_eda.columns))
display(df_eda[[target_col]].head())

Colonnes apres nettoyage: 32


,diagnosis
0,M
1,M
2,M
3,M
4,M


In [7]:
# Distribution des classes (M/B)
class_counts = df_eda[target_col].value_counts(dropna=False).rename_axis('classe').reset_index(name='count')
class_counts['ratio'] = class_counts['count'] / class_counts['count'].sum()
display(class_counts)

fig = px.bar(
    class_counts,
    x='classe',
    y='count',
    text='count',
    color='classe',
    title='Distribution des classes (diagnosis)'
)
fig.update_layout(template='plotly_white', showlegend=False)
fig.show()

,classe,count,ratio
0,B,357,0.627417
1,M,212,0.372583


In [8]:
# Valeurs manquantes
missing = df_eda.isna().sum().sort_values(ascending=False)
missing = missing[missing > 0]

if missing.empty:
    print('Aucune valeur manquante detectee apres nettoyage de base.')
else:
    missing_df = missing.rename_axis('feature').reset_index(name='missing_count')
    display(missing_df.head(20))
    fig = px.bar(
        missing_df.head(20),
        x='feature',
        y='missing_count',
        title='Top 20 des colonnes avec valeurs manquantes'
    )
    fig.update_layout(template='plotly_white', xaxis_tickangle=-45)
    fig.show()

Aucune valeur manquante detectee apres nettoyage de base.


In [9]:
# Separation features numeriques pour stats et correlations
feature_cols = [c for c in df_eda.columns if c not in [target_col, id_col]]
features = df_eda[feature_cols].apply(pd.to_numeric, errors='coerce')

summary = features.describe().T[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]
display(summary.head(10))

,mean,std,min,25%,50%,75%,max
radius_mean,14.127292,3.524049,6.98100,11.70000,13.37000,15.78000,28.11000
texture_mean,19.289649,4.301036,9.71000,16.17000,18.84000,21.80000,39.28000
perimeter_mean,91.969033,24.298981,43.79000,75.17000,86.24000,104.10000,188.50000
area_mean,654.889104,351.914129,143.50000,420.30000,551.10000,782.70000,2501.00000
smoothness_mean,0.096360,0.014064,0.05263,0.08637,0.09587,0.10530,0.16340
compactness_mean,0.104341,0.052813,0.01938,0.06492,0.09263,0.13040,0.34540
concavity_mean,0.088799,0.079720,0.00000,0.02956,0.06154,0.13070,0.42680
concave_points_mean,0.048919,0.038803,0.00000,0.02031,0.03350,0.07400,0.20120
symmetry_mean,0.181162,0.027414,0.10600,0.16190,0.17920,0.19570,0.30400
fractal_dimension_mean,0.062798,0.007060,0.04996,0.05770,0.06154,0.06612,0.09744


In [ ]:
# Correlation (subset des variables les plus correlées à la cible)
label_map = {'M': 1, 'B': 0}
y = df_eda[target_col].map(label_map)
corr_to_target = features.corrwith(y).abs().sort_values(ascending=False)
top_features = corr_to_target.head(12).index.tolist()
corr_matrix = features[top_features].corr()

display(corr_to_target.head(12))

fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu',
    zmin=-1,
    zmax=1,
    title='Matrice de correlation - Top 12 features'
)
fig.update_layout(template='plotly_white')
fig.show()

concave_points_worst    0.793566
perimeter_worst         0.782914
concave_points_mean     0.776614
radius_worst            0.776454
perimeter_mean          0.742636
area_worst              0.733825
radius_mean             0.730029
area_mean               0.708984
concavity_mean          0.696360
concavity_worst         0.659610
compactness_mean        0.596534
compactness_worst       0.590998
dtype: float64

## Conclusion EDA (resume)

- Le dataset est globalement propre, avec peu de traitement structurel necessaire.
- La cible est binaire et exploitable pour un modele de triage.
- Les variables morphologiques montrent des correlations informatives.
- La phase suivante est l'entrainement controle (CV + seuil clinique).